In [1]:
import os
import pysam

In [3]:
def filter_reads_by_chromosome(input_bam, output_directory, output_prefix):
    # Define the list of chromosomes to filter
    target_chromosomes = ['Gus_WT', 'Gus_MUT', 'pEF_GFP']
    genomic_chromosomes = [f'chr{i}' for i in range(1, 23)] + ['chrX', 'chrY']

    # Open the input BAM file
    input_samfile = pysam.AlignmentFile(input_bam, 'rb')

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Extract and set the header for output BAM files
    header = input_samfile.header

    # Initialize output BAM files for target chromosomes
    output_files = {}
    for chromosome in target_chromosomes:
        output_filename = os.path.join(output_directory, f"{output_prefix}_{chromosome}.bam")
        output_files[chromosome] = pysam.AlignmentFile(output_filename, 'wb', header=header)

    # Initialize a single output BAM file for genomic chromosomes
    genomic_output_filename = os.path.join(output_directory, f"{output_prefix}_genomic_reads.bam")
    genomic_output_file = pysam.AlignmentFile(genomic_output_filename, 'wb', header=header)

    # Process each read in the input BAM file
    for read in input_samfile:
        chromosome = read.reference_name

        # Write the read to the corresponding output BAM file
        if chromosome in target_chromosomes:
            output_files[chromosome].write(read)
        elif chromosome in genomic_chromosomes:
            genomic_output_file.write(read)

    # Close all the output BAM files
    for output_file in output_files.values():
        output_file.close()

    genomic_output_file.close()

    # Close the input BAM file
    input_samfile.close()


In [4]:
input_bam_file = "/gscratch/stergachislab/bmallo/large_home/ft_data/plasmid_mix_71923/plasmid_mix_frozen/mixed_plasmids_frozen.bam"
output_directory = "/gscratch/stergachislab/bmallo/large_home/ft_data/plasmid_mix_71923/plasmid_mix_frozen/filtered_bams"
output_prefix = "filtered"
filter_reads_by_chromosome(input_bam_file, output_directory, output_prefix)

ValueError: file has no sequences defined (mode='rb') - is it SAM/BAM format? Consider opening with check_sq=False